In [12]:
import sys
import os
import subprocess
import os
import sys


parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

from utilities_new import *


In [3]:
data_subdir1 = "topocluster_Zee_Large"
data_subdir2 = "topocluster_ZB_EN_only"
accept_data_filename = "l1calo_topocluster_test_Zee.root"
reject_data_filename = "ZMUMU_TopoCluster_Supercell_Large.root"
format_mode = "topocluster_ET_boundaries"
distance_boundaries=[0.00625,0.0125,0.025,0.05,0.1,0.2,0.4]
equalise = False
description = "ring_" + "".join(str(decimal).replace('.', '') for decimal in distance_boundaries)
id1 = f"{data_subdir1}_{description}_eq{equalise}"
id2 = f"{data_subdir2}_{description}"


binary_classifiers = import_all_classifiers()
Classifier = binary_classifiers["ExtraTreesClassifier"]
params = signature(Classifier).parameters
clf = Classifier(random_state=42) if "random_state" in params else Classifier()


X_train, X_test, y_train, y_test, pd_passthrough_train, pd_passthrough_test = prepare_data(accept_data_filename=accept_data_filename, reject_data_filename=reject_data_filename, data_subdir=id1, format_mode=format_mode, distance_boundaries=distance_boundaries)
ZB_X_test, ZB_pd_passthrough_test = prepare_data(accept_data_filename="l1calo_hist_ZB_EN.root",reject_data_filename="l1calo_hist_ZB_EN.root",data_subdir=id2, format_mode=format_mode, distance_boundaries=distance_boundaries, rate_estimation=True)

Imported 43 binary classifiers:
['AdaBoostClassifier', 'BaggingClassifier', 'BernoulliNB', 'CalibratedClassifierCV', 'CategoricalNB', 'ClassifierChain', 'ComplementNB', 'DecisionTreeClassifier', 'DummyClassifier', 'ExtraTreeClassifier', 'ExtraTreesClassifier', 'FixedThresholdClassifier', 'GaussianNB', 'GaussianProcessClassifier', 'GradientBoostingClassifier', 'HistGradientBoostingClassifier', 'KNeighborsClassifier', 'LabelPropagation', 'LabelSpreading', 'LinearDiscriminantAnalysis', 'LinearSVC', 'LogisticRegression', 'LogisticRegressionCV', 'MLPClassifier', 'MultiOutputClassifier', 'MultinomialNB', 'NearestCentroid', 'NuSVC', 'OneVsOneClassifier', 'OneVsRestClassifier', 'OutputCodeClassifier', 'PassiveAggressiveClassifier', 'Perceptron', 'QuadraticDiscriminantAnalysis', 'RadiusNeighborsClassifier', 'RandomForestClassifier', 'RidgeClassifier', 'RidgeClassifierCV', 'SGDClassifier', 'SVC', 'TunedThresholdClassifierCV', 'VotingClassifier', 'XGBClassifier']
found preprepared data in ..\data

100%|██████████| 21761/21761 [00:18<00:00, 1161.75it/s]

selected only one dataset for rate estimation


In [4]:
clf.fit(X_train, y_train)
ZB_y_pred = clf.predict(ZB_X_test)
ZB_pd_passthrough_test["pred"] = ZB_y_pred

In [6]:
event_accept_count = count_accepted(ZB_pd_passthrough_test)
print(event_accept_count)


3910


In [11]:
# path = "C:\Users\Me\AppData\Local\Programs\Python\Python312\python.exe"

python_executable = sys.executable

command = [
    python_executable, "../ManipulateHistos.py",
    "--filesInput=l1calo_topocluster_ZB_EN.root",
    "--analyses", "eFEXEleRateCount"]

try:
    # Run the command and capture the output
    result = subprocess.run(command, capture_output=True, text=True, check=True)

    # Print the output
    print("Output:\n", result.stdout)
    print("Errors:\n", result.stderr)

except subprocess.CalledProcessError as e:
    print("Error running the command:")
    print(e.stderr)

NameError: name 'subprocess' is not defined